# Video Preprocessing

This documentation is used to process video data into images and text to do the prediction that whether the person in video has dementia.


## Part 1: Video to image
This part is to cut the test video into several pictures to do image prediction to test whether the person in video has dementia. The test process is in Documentation 3.3.

In [1]:
!pip install opencv-python 

In [22]:
import cv2
import os
import sys
from moviepy.editor import VideoFileClip
import xlwt

### For dementia test

In [31]:
vc_dem = cv2.VideoCapture('test_dementia.mp4')  # read video file
n = 1  # count number

if vc_dem.isOpened():  # judge whether it can open 
    rval_dem, frame_dem = vc_dem.read()
else:
    rval_dem = False

timeF = 100  # take picture every 'timeF' Frames

i = 0
while rval_dem:  # Loop to read video frames
    rval_dem, frame_dem = vc_dem.read()
    if (n % timeF == 0):  # Store operations every timeF frame
        i += 1
        rows, cols, channel = frame_dem.shape  # Rotate picture if needed
        M = cv2.getRotationMatrix2D((cols/2, rows/2), 180, 1)
        frame_dem = cv2.warpAffine(frame_dem, M, (cols, rows))
        cv2.imwrite('/Users/lyuliangyi/Desktop/STAT8021/STAT8021 Group 2(1)/Documentation 2. Video Preprocessing/figure/test_dementia/image_dem{}.jpg'.format(i), frame_dem)  # 存储为图像
    n = n + 1
    cv2.waitKey(1)
vc_dem.release()

### For normal test

In [32]:
vc_nor = cv2.VideoCapture('test_normal.mp4')  # read video file
n = 1  # count number

if vc_nor.isOpened():  # judge whether it can open 
    rval_nor, frame_nor = vc_nor.read()
else:
    rval_nor = False

timeF = 100  # take picture every 'timeF' Frames

i = 0
while rval_nor:  # Loop to read video frames
    rval_nor, frame_nor = vc_nor.read()
    if (n % timeF == 0):  # Store operations every timeF frame
        i += 1
        rows, cols, channel = frame_nor.shape  # Rotate picture if needed
        M = cv2.getRotationMatrix2D((cols/2, rows/2), 180, 1)
        frame_nor = cv2.warpAffine(frame_nor, M, (cols, rows))
        cv2.imwrite('/Users/lyuliangyi/Desktop/STAT8021/STAT8021 Group 2(1)/Documentation 2. Video Preprocessing/figure/test_normal/image_nor{}.jpg'.format(i), frame_nor)  # 存储为图像
    n = n + 1
    cv2.waitKey(1)
vc_nor.release()

### For dementia video collected from internet

In [41]:
vc_dementia = cv2.VideoCapture('dementia.mp4')  # read video file
n = 1  # count number

if vc_dementia.isOpened():  # judge whether it can open 
    rval_dementia, frame_dementia = vc_dementia.read()
else:
    rval_dementia = False

timeF = 100  # take picture every 'timeF' Frames

i = 0
while rval_dementia:  # Loop to read video frames
    rval_dementia, frame_dementia = vc_dementia.read()
    if (n % timeF == 0):  # Store operations every timeF frame
        i += 1
        rows, cols, channel = frame_dementia.shape  # Rotate picture if needed
        M = cv2.getRotationMatrix2D((cols/2, rows/2), 180, 1)
        frame_dementia = cv2.warpAffine(frame_dementia, M, (cols, rows))
        cv2.imwrite('/Users/lyuliangyi/Desktop/STAT8021/STAT8021 Group 2(1)/Documentation 2. Video Preprocessing/figure/dementia/image_dementia{}.jpg'.format(i), frame_dementia)  # 存储为图像
    n = n + 1
    cv2.waitKey(1)
vc_dementia.release()

## Part 2: Video to audio
This part is to convert the sound from the video to an audio file. 

In [14]:
def convert_video_to_audio_moviepy(video_file, output_ext="wav"):
    
    """Converts video to audio using MoviePy library
    that uses `ffmpeg` under the hood"""
    
    filename, ext = os.path.splitext(video_file)
    clip = VideoFileClip(video_file)
    clip.audio.write_audiofile(f"{filename}.{output_ext}")

# for dementia test
if __name__ == "__main__":
    vf_dem = 'test_dementia.mp4'
    convert_video_to_audio_moviepy(vf_dem)
    
# for normal test 
if __name__ == "__main__":
    vf_nor = 'test_normal.mp4'
    convert_video_to_audio_moviepy(vf_nor)

MoviePy - Writing audio in test_dementia.wav
MoviePy - Done.


MoviePy - Writing audio in test_normal.wav
MoviePy - Done.


## Part 3: Audio to text
This part we do speech recognition to convert the audio to text. We will use the text we got form this part do text prediction to test whether the person in video has dementia in Documentation 4.

In [8]:
!pip install Pydub

In [15]:
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition

def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

### For dementia test

In [16]:
path_dem = 'test_dementia.wav'
text_output_dem = get_large_audio_transcription(path_dem)
print("\nDementia test text:", text_output_dem)

audio-chunks/chunk1.wav : The poem mother's doing dishes. 
audio-chunks/chunk2.wav : There's a boy on the stew. 
audio-chunks/chunk3.wav : Cookie jar in a go down below. 
Error: 
audio-chunks/chunk5.wav : The all you wanted to know. 
audio-chunks/chunk6.wav : Cookie jar. 
audio-chunks/chunk7.wav : The little boy standing on a stool and is ready to go over. 
audio-chunks/chunk8.wav : The little girl standing there the mother's the sink doing dishes. 
audio-chunks/chunk9.wav : What does overflowing that's about it. 

Dementia test text: The poem mother's doing dishes. There's a boy on the stew. Cookie jar in a go down below. The all you wanted to know. Cookie jar. The little boy standing on a stool and is ready to go over. The little girl standing there the mother's the sink doing dishes. What does overflowing that's about it. 


### For normal test

In [17]:
path_nor = 'test_normal.wav'
text_output_nor = get_large_audio_transcription(path_nor)
print("\nNormal test text:", text_output_nor)

audio-chunks/chunk1.wav : The three persons in the kitchen. 
audio-chunks/chunk2.wav : A mother and her two kids. 
audio-chunks/chunk3.wav : The mother is watching fishes for the water coming out of the sink. 
audio-chunks/chunk4.wav : The boy standing on a stool to take cookies from the jar in the cupboard. 
audio-chunks/chunk5.wav : But they still is unsteady and he seems to favour. 
audio-chunks/chunk6.wav : The girl is raising her hand out. 
audio-chunks/chunk7.wav : Nsr finger on hurley. 
audio-chunks/chunk8.wav : Send to ask the boy not to make any noise. 

Normal test text: The three persons in the kitchen. A mother and her two kids. The mother is watching fishes for the water coming out of the sink. The boy standing on a stool to take cookies from the jar in the cupboard. But they still is unsteady and he seems to favour. The girl is raising her hand out. Nsr finger on hurley. Send to ask the boy not to make any noise. 


## Save the text into Excel

In [24]:
# save the text to excel
wb = xlwt.Workbook()
ws = wb.add_sheet('A Test Sheet')
ws.write(0, 0, text_output_dem)

ws.write(1, 0, text_output_nor)
wb.save('test_text.xls')